In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)
from selenium import webdriver
import urllib.request
import urllib.parse
import time
import re #정규식

from selenium.webdriver.common.keys import Keys
import pyautogui

## 강좌 크롤링 

In [7]:
url = 'https://everytime.kr/timetable'
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get(url)
time.sleep(3)
user_id = '본인_아이디_입력' # input('에브리타임 아이디를 입력해주세요.: ')
et_login = driver.find_element_by_name("userid")
et_login.clear()
et_login.send_keys(user_id)

user_pw = '본인_비밀번호_입력'  # input('에브리타임 비밀번호를 입력해주세요.: ')
et_login = driver.find_element_by_name("password")
et_login.clear()
et_login.send_keys(user_pw) #자신의 비번을 넣으세요

driver.find_element_by_xpath("""//*[@id="container"]/form/p[3]/input""").click()
time.sleep(3)
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="sheet"]/ul/li[3]/a""").click()
        time.sleep(2)
        break
    except:
        continue
timetable_all = pd.DataFrame()
timetable_all
graph = {}
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="container"]/ul/li[1]""").click()
        time.sleep(2)
        break
    except:
        continue
        
# 여기서 range를 조절해서 학과 선택
for i in range(1):
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="subjects"]/div[1]/a[4]""").click()
            time.sleep(2)
            break
        except:
            continue
    # 항상 range 에서 가장 먼저 돌아가는 숫자 넣어주기
    while i == 1:
        try:         
            # xpath 안쪽에 div/ul/li[n] 여기서 n 을 조절해서 대학선택 ( n = 1 -> 외국어 대학, n = 2 -> 공과대학...)
            driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/li[2]""").click()
            time.sleep(2)
            break
        except:
            continue
    while True:
        try:
            # xpath 안쪽에 div/ul/ul[n] 여기서 n 을 조절해서 대학선택 ( n = 1 -> 외국어 대학, n = 2 -> 공과대학...)
            # 옆에 있는 str (i + 1 )의 경우 각 대학안에서 학과 조절 --> 위에 for 반복문과 연계
            driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/li["""+str((i+1))+"""]""").click()
            time.sleep(2)
            break
        except:
            continue
            
"""  
    #스크롤 맨아래로 내리기
    
    pre_count = 0
    while True:
     #   #tr요소 접근
        element = driver.find_elements_by_css_selector("#subjects > div.list > table > tbody > tr")

        # tr 마지막 요소 접근
        result = element[-1]
        #마지막요소에 focus주기
        driver.execute_script('arguments[0].scrollIntoView(true);',result)
        sleep(2)

        #현재 접근한 요소의 갯수
        current_count = len(element)
        if pre_count == current_count:
            break
            
        #같지않다면
        pre_count = current_count
        
"""
            
        
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    contents = []
    tmp = soup.find('div','list').find('tbody').find_all('td')
 

    
    for i in tmp:
        
        tmp = i.get_text()
        
        # 별점 정보와 해쉬태그 정보를 얻기 위해서 과목 상세페이지로 이동
        if (str(i)[14:18]=='star'):
            address = i.find('a')['href']
            url = 'https://everytime.kr' + str(address)
            
            
            # 새로운 탭을 열고 해당과목 상세페이지로 이동 
            pyautogui.hotkey('ctrl','t')
            time.sleep(0.2)
            driver.switch_to_window(driver.window_handles[1])
            driver.get(url)
            time.sleep(0.5)
            page = driver.page_source
            soup = BeautifulSoup(page, "html.parser")
            # 별점
            RateArea = soup.select('#container > div.side.article > div.rating > div.rate > span > span.value')
            contents.append(RateArea[0].text)
            
            # 해쉬 태그들 - 과제, 조모임, 학점비율, 출결, 시험횟수
            HashTags = soup.select('#container > div.side.article > div.rating > div.details > p > span')  
            
            # 해쉬태그에 관한 정보 없을 시
            if len(HashTags) == 0:
                for t in range(5):
                    contents.append('정보 없음')
                
            # 해쉬태그에 관한 정보 존재 시
            else:
                for HashTag in HashTags:
                    contents.append(HashTag.text)
            
            
            # 탭을 다시 닫음
            driver.close()
            # time.sleep(0.5)

            
            # 첫번째 탭으로 전환
            driver.switch_to_window(driver.window_handles[0])
            # time.sleep(0.5)
            
            continue
            
        contents.append(tmp)   
    contents = np.array(contents)
    
    
    
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    column = []
    tmp = soup.find('div','list').find('thead').find_all('th')
    for i in tmp:
        tmp = str(i).lstrip('<th>').split('<div>')
        
        # 해쉬태그용 항목들
        if str(tmp[0]) == '강의평':
            column.append(tmp[0])
            HashTagHeads = ['과제', '조모임', '학점 비율', '출결', '시험횟수']
            for HashTagHead in HashTagHeads:
                column.append(HashTagHead)
         
        else:
            column.append(tmp[0])    
        
    
    # 17개의 열로 다시 정렬
    timetable = pd.DataFrame(contents.reshape(len(contents)//17,-1), columns=column)
    
    
    
    major = soup.find('a','item active').get_text().split(':')[1]
    graph[str(major)] = len(timetable)
    timetable_all = pd.concat([timetable_all,timetable],ignore_index=True)
    timetable.to_excel('./data/20년 2학기 '+major+' 강좌.xlsx',encoding = 'utf8',sheet_name=major,index = False) # 학과 별로 저장
time.sleep(2)
driver.close()




NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=87.0.4280.66)


## 해쉬태그용 크롤링 

In [28]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse
import webbrowser
from selenium.webdriver.common.keys import Keys
import pyautogui


# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome(executable_path=r'./driver/chromedriver')

# url에 접근한다.
driver.get('https://everytime.kr/login')

# 암묵적으로 웹 자원 로드를 위해 5초까지 기다려 준다.
driver.implicitly_wait(5)

# 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('userid').send_keys('--본인_아이디_입력--')
driver.find_element_by_name('password').send_keys('--본인_비밀번호_입력--')

# 로그인 버튼을 눌러주자.
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

# 예시 사이트에 접근
driver.get('https://everytime.kr/lecture/view/1854171')

# 탭하나 추가 실행
pyautogui.hotkey('ctrl','t')
time.sleep(2)
print(driver.window_handles)
driver.switch_to_window(driver.window_handles[1])
driver.get('https://everytime.kr/lecture/view/1854171')


# 현재 탭 종료
driver.close()
time.sleep(2)


# 첫번째 탭으로 전환
driver.switch_to_window(driver.window_handles[0])

# 별점 정보 crawl
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")
RateArea = soup.select('#container > div.side.article > div.rating > div.rate > span > span.value')
print(RateArea[0].text)

# 강의 정보 crawl
HashTags = soup.select('#container > div.side.article > div.rating > div.details > p > span')
#container > div.side.article > div.rating > div.details > p:nth-child(1) > span
for HashTag in HashTags:
    print(HashTag.text)

['CDwindow-DE3C79D394F0B37616A4AB8018EBFD69', 'CDwindow-9147CE18C2D31124CF19E178DB34FC26']


C:\Users\kjeng\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: use driver.switch_to.window instead
C:\Users\kjeng\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: use driver.switch_to.window instead


4
많음
많음
비율 채워줌
직접호명
두 번


## 파일들 확인  

In [36]:
timetable.head()

,대상학년,강좌코드,강좌명,교수명,학점,강의시간/강의실,이수구분,담은 인원,정원,특이사항
0,2,ELC204-00,Critical Reading and Writing 2,Todd Christopher Mcgowan,3,월 09:00-10:15 (외305)수 09:00-10:15 (외305),전공필수(04),30,0,영어
1,2,ELC204-01,Critical Reading and Writing 2,Todd Christopher Mcgowan,3,월 16:30-17:45 (외305)수 16:30-17:45 (외305),전공필수(04),28,0,영어
2,2,ELC204-02,Critical Reading and Writing 2,Todd Christopher Mcgowan,3,화 09:00-10:15 (외412)목 09:00-10:15 (외412),전공필수(04),22,0,영어
3,2,ELC203-01,Speech and Discussion 2,Peggy Cho,3,금 13:30-16:15 (외305),전공필수(04),25,0,영어
4,2,ELC203-02,Speech and Discussion 2,Todd Christopher Mcgowan,3,월 12:00-13:15 (외402)수 12:00-13:15 (외402),전공필수(04),23,0,영어


In [35]:
timetable.tail()

,대상학년,강좌코드,강좌명,교수명,학점,강의시간/강의실,이수구분,담은 인원,정원,특이사항
45,1,ESP102-00,스페인어A2문법,김찬기,3,화 15:00-16:15 (외201)목 15:00-16:15 (외201),전공선택(05),30,0,
46,1,ESP102-01,스페인어A2문법,권미선,3,화 15:00-16:15 (외202)목 15:00-16:15 (외202),전공선택(05),40,0,
47,1,ESP106-00,스페인어A2회화,김보영 / Armando Daniel Bazan,3,월 13:30-14:45 (외201)수 13:30-14:45 (외201),전공선택(05),20,0,팀티칭
48,1,ESP106-01,스페인어A2회화,김보영 / Armando Daniel Bazan,3,월 13:30-14:45 (외211)수 13:30-14:45 (외211),전공선택(05),18,0,팀티칭
49,1,ESP106-02,스페인어A2회화,김보영 / Armando Daniel Bazan,3,월 15:00-16:15 (외201)수 15:00-16:15 (외201),전공선택(05),20,0,팀티칭


In [40]:
timetable = pd.read_excel('./data/20년 2학기 환경학및환경공학과 강좌.xlsx') 
timetable.head()

,계획서,대상학년,강좌코드,강좌명,교수명,학점,강의시간/강의실,이수구분,강의평,과제,조모임,학점 비율,출결,시험횟수,담은 인원,정원,특이사항
0,조회,23,ENV386-00,수질오염조사실험및설계,김정수,3,월 15:00-19:50 (공102-1호),전공필수(04),0.00,정보 없음,정보 없음,정보 없음,정보 없음,정보 없음,15,0,NaN
1,조회,23,ENV386-01,수질오염조사실험및설계,유창규,3,수 15:00-17:45 (공102-1호)수 18:00-19:50 (공340호),전공필수(04),3.00,많음,많음,학점느님,지정좌석,한 번,19,0,NaN
2,조회,4,ENV487-00,졸업논문(환경학및환경공학전공),유가영,0,토 09:00-10:15 (공369호),전공필수(04),0.00,정보 없음,정보 없음,정보 없음,정보 없음,정보 없음,15,0,마지막 학기 졸업예정자 필수과목
3,조회,1,ENV174-01,환경과학개론,황선진,3,화 15:00-16:15 (공369호)목 15:00-16:15 (공369호),전공필수(04),3.83,보통,없음,비율 채워줌,직접호명,두 번,54,0,교직과목
4,조회,34,ENV418-00,환경종합설계,유창규,3,월 16:30-19:15 (공369호),전공필수(04),0.00,정보 없음,정보 없음,정보 없음,정보 없음,정보 없음,35,0,NaN


In [39]:
timetable = pd.read_excel('./data/20년 2학기 환경학및환경공학과 강좌.xlsx') 
timetable.tail()

,계획서,대상학년,강좌코드,강좌명,교수명,학점,강의시간/강의실,이수구분,강의평,과제,조모임,학점 비율,출결,시험횟수,담은 인원,정원,특이사항
16,조회,1,ENV273-01,공학수학1,민부기,3,화 13:30-14:45 (공369)목 13:30-14:45 (공369),전공기초(11),3.25,없음,없음,학점느님,직접호명,두 번,45,0,"영어(부분), 환경과 학생만 수강가능"
17,조회,2,ENV274-00,공학프로그래밍입문,장광현,3,화 10:30-11:45 (공176-4호)목 10:30-11:45 (공176-4호),전공기초(11),4.29,보통,없음,비율 채워줌,직접호명,세 번,12,0,전공기초 환경과 전공학생만 수강
18,조회,2,ENV274-01,공학프로그래밍입문,장광현,3,화 12:00-13:15 (공176-4호)목 12:00-13:15 (공176-4호),전공기초(11),4.29,보통,없음,비율 채워줌,직접호명,세 번,22,0,전공기초 환경과 전공학생만 수강
19,조회,3,EDU3334-G00,교과교수법(환경),유가영,3,금 12:00-14:45 (공369호),교직전선(20),0.00,정보 없음,정보 없음,정보 없음,정보 없음,정보 없음,1,0,교직과목(교직학생만 수강가능)
20,조회,3,EDU3190-01,교과교재연구및지도법(환경),유가영,3,금 15:00-17:45 (공369),교직전선(20),0.00,정보 없음,정보 없음,정보 없음,정보 없음,정보 없음,1,0,교직과목(교직학생만 수강가능)


## 전체 강좌 크롤링

In [ ]:
#-*- coding:utf-8 -*-
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from openpyxl import Workbook,load_workbook

# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome(executable_path=r'./driver/chromedriver')

# url에 접근한다.
driver.get('https://everytime.kr/login')
# 암묵적으로 웹 자원 로드를 위해 5초까지 기다려 준다.
driver.implicitly_wait(5)

# 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('userid').send_keys('본인_아이디_입력')
driver.find_element_by_name('password').send_keys('본인_비밀번호_입력 ')

# 로그인 버튼을 눌러주자.
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

driver.get('https://everytime.kr/timetable')

#수업 목록에서 검색 클릭
driver.find_element_by_xpath('//*[@id="container"]/ul/li[2]').click()

#팝업창 닫기
sleep(4)
driver.find_element_by_xpath('//*[@id="sheet"]/ul/li[3]/a').click()

pre_count = 0
#스크롤 맨아래로 내리기
while True:
    #tr요소 접근
    element = driver.find_elements_by_css_selector("#subjects > div.list > table > tbody > tr")

    # tr 마지막 요소 접근
    result = element[-1]
    #마지막요소에 focus주기
    driver.execute_script('arguments[0].scrollIntoView(true);',result)
    sleep(2)

    #현재 접근한 요소의 갯수
    current_count = len(element)
    if pre_count == current_count:
        break
    #같지않다면
    pre_count = current_count


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

trs = soup.select('#subjects > div.list > table > tbody > tr')

results = []

for tr in trs:
    result=[]
    tds = tr.select('#subjects > div.list > table > tbody > tr > td')
    result.append(tds[0].text) #조회
    result.append(tds[1].text) #대상학년
    result.append(tds[2].text) #학수번호
    result.append(tds[3].text) #강의이름
    result.append(tds[4].text) #교수님성함
    result.append(tds[5].text) #학점
    result.append(tds[6].text) #수업시간
    result.append(tds[7].text) #강의유형
    result.append(tds[9].text) #담은 인원
    result.append(tds[11].text) # 특이사항
    results.append(result)

if results:
    print("성공!!")
excel_column = 10
write_wb = Workbook()
write_ws = write_wb.create_sheet('result.xls')
for data in results:
    write_ws = write_wb.active
    write_ws.append(data)
write_wb.save('C:/Users/kjeng/강의평 크롤링/result.csv')
